In [ ]:
import numpy as np
import shelve

## Importing 1st stage results

In [ ]:
filename = r'bbl.out'

c_list = [899, 2616, 6586, 24763, 30155, 30352] # list of movie theater ids

var_dict = {}

with shelve.open(filename) as ws:
    for c in c_list:
        var_dict[f'{c}_complete'] = ws[f'{c}_completo_add_noise'].astype('float64') # just plug in chosen file

## Objective Function

In [ ]:
# takes as args 2-parameter guesses and a tuple with (c_list, stored_results)

def objective(params, *args):
    loss = 0 # loss value to be minimized
    par_array = np.array([params[0],1,params[1]]) # parameter arrays, 2nd parameter value (private shocks) is normalized to 1
    for c in args[0]:
        orig = np.sum(np.dot(args[1][f'{c}_complete'][:,:,0],par_array)) # cval, or original value func
        loss += np.square([max(0, np.dot(var_dict['899_complete'][:,:,i],par_array).sum() - orig) for i in range(
            1,var_dict['899_complete'].shape[2])]).sum() # takes the square of the max of o or the difference between 
                                                            # bias/noise distvals and cval
    return loss

## Minimizing KDE values
Tinkering with guesses and optimization algorithm

In [ ]:
from scipy.optimize import minimize

In [ ]:
for i in range(10):
    print(minimize(objective, np.array([50*i,50*i]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500}).x)

In [ ]:
results = minimize(objective2, np.array([0,0]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500})

In [ ]:
results.x

In [ ]:
print(np.square([results.hess_inv[0,0], results.hess_inv[1,1]]))

## Minimizing Logit values
Tinkering with guesses and optimization algorithm

In [ ]:
minimize(objective, np.array([0,0]), args=(c_list, var_dict), method='Nelder-Mead', options={'maxiter':500})

In [ ]:
for i in range(10):
    print(minimize(objective, np.array([50*i,-50*i]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500}).x)

## Results and Hessian Inverse

In [ ]:
results = minimize(objective, np.array([0,0]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500})

In [ ]:
results.x

In [ ]:
print(np.sqrt([results.hess_inv[0,0], results.hess_inv[1,1]]))